In [78]:
import pickle
import os
import datetime
import pandas as pd
from IPython.core.display import display
from config.appconfig import config
from plotly.subplots import make_subplots
import plotly.io as pio
pio.kaleido.scope.default_scale = 16

# Import data from data directories
emcb_appliance_file_names = [f for f in os.listdir(config.PROCESSED_EMCB_DATA_DIR)
                             if os.path.isfile(os.path.join(config.PROCESSED_EMCB_DATA_DIR, f))]
ted_appliance_file_names = [f for f in os.listdir(config.PROCESSED_TED_DATA_DIR)
                            if os.path.isfile(os.path.join(config.PROCESSED_TED_DATA_DIR, f))]

emcb_appliance_df_dict = dict()
for appliance_name in emcb_appliance_file_names:
    file = open(config.PROCESSED_EMCB_DATA_DIR + '\\' + appliance_name, 'rb')
    loaded_appliance_data = pickle.load(file)
    loaded_appliance_data.reset_index(drop=True, inplace=True)
    emcb_appliance_df_dict[appliance_name.replace('.pkl','')] = loaded_appliance_data

ted_appliance_df_dict = dict()
for appliance_name in ted_appliance_file_names:
    file = open(config.PROCESSED_TED_DATA_DIR + '\\' + appliance_name, 'rb')
    loaded_appliance_data = pickle.load(file)
    loaded_appliance_data.reset_index(drop=True, inplace=True)
    loaded_appliance_data['Time'] = pd.to_datetime(loaded_appliance_data['Time'])
    loaded_appliance_data.sort_values(by='Time', inplace=True, ascending=True)
    ted_appliance_df_dict[appliance_name.replace('.pkl','')] = loaded_appliance_data

file = open(config.PROCESSED_TRANE_DATA_DIR + '\\' + 'trane_energy_df.pkl', 'rb')
trane_energy_df = pickle.load(file)

In [79]:
# Calculate overall heat pump energy consumption using TED data
ted_heat_pump_df = pd.concat(
    [ted_appliance_df_dict['Air_Handling_Unit'],
     ted_appliance_df_dict['Heat_Pump_Heater'],
     ted_appliance_df_dict['Outdoor_Unit']]).groupby('Time').sum().reset_index()
ted_heat_pump_df['Time'] = pd.to_datetime(ted_heat_pump_df['Time'])
ted_heat_pump_df = ted_heat_pump_df.sort_values(by=['Time'])


In [80]:
# Display heat pump Energy over the time period
import plotly.graph_objects as go

data_name = 'Daily Measured Real Energy Consumed [kWh]'
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=emcb_appliance_df_dict['AC_Compressor']['Date'],
    y=emcb_appliance_df_dict['AC_Compressor'][data_name],
    name=data_name,
    line=dict(color='royalblue', width=2)))

filtered_df = trane_energy_df[trane_energy_df['Data'].between(
    pd.to_datetime(emcb_appliance_df_dict['AC_Compressor']['Date']).iloc[-1],
    pd.to_datetime(ted_appliance_df_dict['Outdoor_Unit']['Time']).iloc[-1])]

fig.add_trace(go.Scatter(
    x=filtered_df['Data'],
    y=filtered_df['Energy [kWh]'],
    name=data_name,
    line=dict(color='royalblue', width=2)))

filtered_df = ted_appliance_df_dict['Outdoor_Unit'][pd.to_datetime(ted_appliance_df_dict['Outdoor_Unit']['Time'])
                                    < datetime.datetime(2020, 6, 1)]

fig.add_trace(go.Scatter(
    x=pd.to_datetime(filtered_df['Time']),
    y=filtered_df['Energy [kWh]'],
    name=data_name,
    line=dict(color='royalblue', width=2)))

fig.update_layout(xaxis_title="Month, Year", yaxis_title="Energy [kWh]", title="DC House Heat Pump Daily Energy Data",
                  template="none", showlegend=False)
fig.update_xaxes(tickangle=-45, tickvals=pd.date_range('2018-03-22', '2020-06', freq='QS'), rangeslider_visible=False)
fig.show()
fig.write_image(config.IMAGE_DIR + "\heat_pump_energy.png")

In [102]:
# Combine heat pump energy vs weather data
hourly_filename = '2018-03-22_to_2020-06-02_hourly_weather_data.pkl'
file = open(config.PROCESSED_WEATHER_DATA_DIR + '\\' + hourly_filename, 'rb')
loaded_hourly_data = pickle.load(file)
loaded_hourly_data.reset_index(drop=True, inplace=True)

data_name = 'Daily Measured Real Energy Consumed [kWh]'
fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])

filtered_trane_df = trane_energy_df[trane_energy_df['Data'].between(
    pd.to_datetime(emcb_appliance_df_dict['AC_Compressor']['Date']).iloc[-1],
    pd.to_datetime(ted_appliance_df_dict['Outdoor_Unit']['Time']).iloc[0])]
filtered_trane_df.sort_values(by='Data', inplace=True, ascending=True)

filtered_ted_df = ted_appliance_df_dict['Outdoor_Unit'][ted_appliance_df_dict['Outdoor_Unit']['Time']
                                                        < datetime.datetime(2020, 6, 1)]
filtered_ted_df.sort_values(by='Time', inplace=True, ascending=True)

heat_pump_time = pd.to_datetime(
    pd.concat(
        [emcb_appliance_df_dict['AC_Compressor']['Date'],
         filtered_trane_df['Data'],
         filtered_ted_df['Time']]
    )
)

heat_pump_data = pd.concat(
    [emcb_appliance_df_dict['AC_Compressor'][data_name],
     filtered_trane_df['Energy [kWh]'],
     filtered_ted_df['Energy [kWh]']]
)


fig.add_trace(go.Scatter(
    x=heat_pump_time,
    y=heat_pump_data,
    name=data_name))
    #line=dict(color='red', width=2)))

fig.add_trace(
    go.Scatter(
        x=loaded_hourly_data.time,
        y=loaded_hourly_data.temperature,
        name="Outdoor Temperature [F]"),
        #line=dict(color='royalblue', width=2)),
    secondary_y=True
)

fig.update_layout(xaxis_title="Month, Year", title="DC House Historical Weather Data",
                  template="seaborn", legend=dict(
                                                orientation="h",
                                                yanchor="bottom",
                                                y=1.02,
                                                xanchor="right",
                                                x=1.01))
fig.update_xaxes(tickangle=-45, tickvals=pd.date_range('2018-03-22', '2020-06', freq='QS'), rangeslider_visible=False)
fig.update_yaxes(title_text="<b>Energy [kWh]<b>", secondary_y=False, color='#6080b7')
fig.update_yaxes(title_text="<b>Temperature [F]<b>", secondary_y=True, color='#da895d')
fig.show()
fig.write_image(config.IMAGE_DIR + "\heat_pump_energy_vs_weather.png")

<ipython-input-102-1c2081b03efc>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-102-1c2081b03efc>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [82]:
# Display heat pump Real Power over the time period
data_name = 'Daily Measured Real Power (avg) [kW]'
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=emcb_appliance_df_dict['AC_Compressor']['Date'],
    y=emcb_appliance_df_dict['AC_Compressor'][data_name],
    name=data_name))
data_name = 'Daily Calculated Real Power (max) [kW]'
fig.add_trace(go.Scatter(
    x=emcb_appliance_df_dict['AC_Compressor']['Date'],
    y=emcb_appliance_df_dict['AC_Compressor'][data_name],
    name=data_name))
fig.update_layout(xaxis_title="Month, Year", yaxis_title="Power [kW]", title="Heat Pump Historical Real Power Data",
                  template="none", legend=dict(
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1.02,
                                    xanchor="right",
                                    x=1.02))
fig.show()


In [83]:
# Display heat pump Reactive Power over the time period
data_name = 'Daily Calculated Reactive Power (avg) [kVAR]'
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=emcb_appliance_df_dict['AC_Compressor']['Date'],
    y=emcb_appliance_df_dict['AC_Compressor'][data_name],
    name=data_name))
data_name = 'Daily Calculated Reactive Power (max) [kVAR]'
fig.add_trace(go.Scatter(
    x=emcb_appliance_df_dict['AC_Compressor']['Date'],
    y=emcb_appliance_df_dict['AC_Compressor'][data_name],
    name=data_name))
fig.update_layout(xaxis_title="Month, Year", yaxis_title="Power [kVAR]",
                  title="Heat Pump Historical Reactive Power Data",
                  template="none", legend=dict(
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1.02,
                                    xanchor="right",
                                    x=1.02))
fig.show()

In [84]:
# Display heat pump Apparent Power over the time period
data_name = 'Daily Calculated Apparent Power (avg) [kVA]'
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=emcb_appliance_df_dict['AC_Compressor']['Date'],
    y=emcb_appliance_df_dict['AC_Compressor'][data_name],
    name=data_name))
data_name = 'Daily Calculated Apparent Power (max) [kVA]'
fig.add_trace(go.Scatter(
    x=emcb_appliance_df_dict['AC_Compressor']['Date'],
    y=emcb_appliance_df_dict['AC_Compressor'][data_name],
    name=data_name))
fig.update_layout(xaxis_title="Month, Year", yaxis_title="Power [kVA]",
                  title="Heat Pump Historical Apparent Power Data",
                  template="none", legend=dict(
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1.02,
                                    xanchor="right",
                                    x=1.02))
fig.show()

In [85]:
# Display heat pump Power Factor over the time period
data_name = 'Daily Measured Power Factor (avg) [-]'
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=emcb_appliance_df_dict['AC_Compressor']['Date'],
    y=emcb_appliance_df_dict['AC_Compressor'][data_name],
    name=data_name))
data_name = 'Daily Measured Power Factor (max) [-]'
fig.add_trace(go.Scatter(
    x=emcb_appliance_df_dict['AC_Compressor']['Date'],
    y=emcb_appliance_df_dict['AC_Compressor'][data_name],
    name=data_name))
fig.update_layout(xaxis_title="Month, Year", yaxis_title="Power Factor [-]",
                  title="Heat Pump Historical Power Factor Data",
                  template="none", legend=dict(
                                    orientation="h",
                                    yanchor="bottom",
                                    y=1.02,
                                    xanchor="right",
                                    x=1.02))
fig.show()